To automatically predict the congestion level on-board of public transport via open data eg. Trafiklab/Google and data pipeline powered by IBM and other open source community 

Step 1. Read the ID from station names

In [ ]:
import requests 

url1 = "https://api.resrobot.se/v2/location.name?key=a4961aa1-04e7-42fa-9aa2-0d373e901a01&input=Centralen"
response = requests.get(url1)

print(response.status_code)


In [23]:
#print(response.content)
import pandas as pd
import xml.etree.ElementTree as ET
from lxml.etree import fromstring

root = ET.fromstring(response.content)
for child in root.iter('{hafas_rest_v1}StopLocation'):
    print(child.attrib['id'],child.attrib['name'])

dfcols = ['name','ID','lon','lat']
df = pd.DataFrame(columns = dfcols)

for child in root.iter('{hafas_rest_v1}StopLocation'):
    name = child.attrib['name']
    ID = child.attrib['id']
    lon = child.attrib['lon']
    lat = child.attrib['lat']
    df = df.append(pd.Series([name, ID, lon,lat], index = dfcols), ignore_index = True)

df.head()

740020749 T-Centralen T-bana (Stockholm kn)
740035997 T-Centralen Spårv (Stockholm kn)
740001360 Töcksfors Centralen (Årjäng kn)
740049537 Centralplan (Hallsberg kn)
740051308 Centralplan (Karlskoga kn)
740066254 Centralplan (Nacka kn)
740069144 Centralvägen (Huddinge kn)
740069868 Centralvägen (Södertälje kn)
740050578 Centralvägen Frövi (Lindesberg kn)
740037247 Centrallager nord (Älmhult kn)


,name,ID,lon,lat
0,T-Centralen T-bana (Stockholm kn),740020749,18.059266,59.330945
1,T-Centralen Spårv (Stockholm kn),740035997,18.062133,59.332131
2,Töcksfors Centralen (Årjäng kn),740001360,11.841409,59.508572
3,Centralplan (Hallsberg kn),740049537,15.338402,59.065691
4,Centralplan (Karlskoga kn),740051308,14.515179,59.326019


step 2 To get public transport (realtime info) for arrival and departure time at the searched ID

In [24]:
url2 = "https://api.resrobot.se/v2/departureBoard?key=53bd14d9-352e-4d20-853f-0ac75f1796ae&id=740020749"
response2 = requests.get(url2)



200
b'<?xml version="1.0" encoding="UTF-8"?>\n<LocationList xmlns="hafas_rest_v1">\n  <StopLocation id="740020749" extId="740020749" name="T-Centralen T-bana (Stockholm kn)" lon="18.059266" lat="59.330945" weight="17843" products="32"/>\n  <StopLocation id="740035997" extId="740035997" name="T-Centralen Sp\xc3\xa5rv (Stockholm kn)" lon="18.062133" lat="59.332131" weight="2406" products="192"/>\n  <StopLocation id="740001360" extId="740001360" name="T\xc3\xb6cksfors Centralen (\xc3\x85rj\xc3\xa4ng kn)" lon="11.841409" lat="59.508572" weight="584" products="640"/>\n  <StopLocation id="740049537" extId="740049537" name="Centralplan (Hallsberg kn)" lon="15.338402" lat="59.065691" weight="548" products="128"/>\n  <StopLocation id="740051308" extId="740051308" name="Centralplan (Karlskoga kn)" lon="14.515179" lat="59.326019" weight="548" products="128"/>\n  <StopLocation id="740066254" extId="740066254" name="Centralplan (Nacka kn)" lon="18.233531" lat="59.315897" weight="548" products="128"

In [25]:
print(response2.status_code)
print(response2.content)

200
b'<?xml version="1.0" encoding="UTF-8"?>\n<DepartureBoard xmlns="hafas_rest_v1">\n  <Departure direction="Farsta Strand T-bana (Stockholm kn)" name="L\xc3\xa4nstrafik -Tunnelbana 18" transportNumber="18" transportCategory="ULT" type="S" stopid="740020749" stopExtId="740020749" stop="T-Centralen T-bana (Stockholm kn)" date="2020-05-02" time="18:55:00">\n    <Product catCode="5" catOutL="L\xc3\xa4nstrafik -Tunnelbana" catOutS="ULT" name="L\xc3\xa4nstrafik -Tunnelbana 18" num="18" operator="SL" operatorCode="275" operatorUrl="http://www.sl.se"/>\n    <Stops>\n      <Stop depDate="2020-05-02" depTime="18:55:00" routeIdx="8" name="T-Centralen T-bana (Stockholm kn)" id="740020749" extId="740020749" lon="18.059266" lat="59.330945"/>\n      <Stop arrDate="2020-05-02" arrTime="18:57:00" routeIdx="9" name="Gamla Stan T-bana (Stockholm kn)" id="740021653" extId="740021653" lon="18.067617" lat="59.32316"/>\n      <Stop arrDate="2020-05-02" arrTime="18:58:00" routeIdx="10" name="Slussen T-bana 

In [30]:
root2 = ET.fromstring(response2.content)
for child in root2.iter('{hafas_rest_v1}Departure'):
    print(child.tag,child.attrib)

{hafas_rest_v1}Departure {'direction': 'Farsta Strand T-bana (Stockholm kn)', 'name': 'Länstrafik -Tunnelbana 18', 'transportNumber': '18', 'transportCategory': 'ULT', 'type': 'S', 'stopid': '740020749', 'stopExtId': '740020749', 'stop': 'T-Centralen T-bana (Stockholm kn)', 'date': '2020-05-02', 'time': '18:55:00'}
{hafas_rest_v1}Departure {'direction': 'Hjulsta T-bana (Stockholm kn)', 'name': 'Länstrafik -Tunnelbana 10', 'transportNumber': '10', 'transportCategory': 'ULT', 'type': 'S', 'stopid': '740020749', 'stopExtId': '740020749', 'stop': 'T-Centralen T-bana (Stockholm kn)', 'date': '2020-05-02', 'time': '18:55:00'}
{hafas_rest_v1}Departure {'direction': 'Kungsängen station (Upplands Bro kn)', 'name': 'Pendeltåg linje 43', 'transportNumber': '43', 'transportCategory': 'JLT', 'type': 'S', 'stopid': '740001617', 'stopExtId': '740001617', 'stop': 'Stockholm City station', 'date': '2020-05-02', 'time': '18:56:00'}
{hafas_rest_v1}Departure {'direction': 'Alvik T-bana (Stockholm kn)', 'n

In [31]:
dfcols2 = ['name','stop','stopid','direction','transportNumber','date','time']
df2 = pd.DataFrame(columns = dfcols2)

for child in root2.iter('{hafas_rest_v1}Departure'):
    lineName = child.attrib['name']
    stopName = child.attrib['stop']
    stopID = child.attrib['stopid']
    lineDirection = child.attrib['direction']
    lineNumber = child.attrib['transportNumber']
    date = child.attrib['date']
    time = child.attrib['time']    
    df2 = df2.append(pd.Series([lineName,stopName,stopID,lineDirection,lineNumber,date,time], index = dfcols2), ignore_index = True)

df2.head()

,name,stop,stopid,direction,transportNumber,date,time
0,Länstrafik -Tunnelbana 18,T-Centralen T-bana (Stockholm kn),740020749,Farsta Strand T-bana (Stockholm kn),18,2020-05-02,18:55:00
1,Länstrafik -Tunnelbana 10,T-Centralen T-bana (Stockholm kn),740020749,Hjulsta T-bana (Stockholm kn),10,2020-05-02,18:55:00
2,Pendeltåg linje 43,Stockholm City station,740001617,Kungsängen station (Upplands Bro kn),43,2020-05-02,18:56:00
3,Länstrafik -Tunnelbana 19,T-Centralen T-bana (Stockholm kn),740020749,Alvik T-bana (Stockholm kn),19,2020-05-02,18:56:00
4,Länstrafik - Buss 69,Stockholm C Klarabergsviadukte,740004383,Stockholm Kaknästornet,69,2020-05-02,18:57:00
